In [ ]:
from tkinter import *
import PIL
from PIL import Image, ImageDraw
import numpy as np
from tensorflow.keras.models import load_model
import cv2
import os

#uses cpu when there is gpu available
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [ ]:
model = load_model('model.h5')

root = Tk()

#canvas width and height
h,w = 200,200
lastx, lasty = None, None

def predict():

    X,Y = calculate_center(image1)
    image2 = zoom_at(image1,X,Y,1.8)

    #convert to cv2 from PIL
    a = cv2.resize(np.array(image2),(28,28), interpolation=Image.LANCZOS)
    a = np.expand_dims(a, axis=-1)
    
    #set the label as the model predicts
    var.set(str(np.argmax(model.predict(np.array([a])))))
            
    

#this function will calculate the center of the points drawn so we can zoom into drawn part 
def calculate_center(img):
    n=0
    x = 0
    y = 0
    for i in range(w):
        for j in range(h):
            if img.getpixel((i,j)) == 255:
                n += 1
                x += i
                y += j
    return (x//n,y//n)

#zooming is necessary because the model was trained on centered images and it causes wrong predictions when the number drawn is not at center
def zoom_at(img, x, y, zoom):
    w, h = img.size
    zoom2 = zoom * 2
    img = img.crop((x - w / zoom2, y - h / zoom2, 
                    x + w / zoom2, y + h / zoom2))
    return img.resize((w, h), Image.LANCZOS)


def reset():
    cv.delete("all")
    draw.rectangle((0, 0, w, h), fill=0)
    var.set(' ')
    

def activate_paint(e):
    global lastx, lasty
    cv.bind('<B1-Motion>', paint)
    lastx, lasty = e.x, e.y
    


def paint(e):
    global lastx, lasty,image1,counter
    
    if lastx and lasty:
        x, y = e.x, e.y
        
        counter += 1
                
        cv.create_line((lastx, lasty, x, y),fill='white', width=8)
        draw.line((lastx, lasty, x, y), fill='white', width=10)
        
        predict()
        
        
        
    lastx, lasty = x, y





#tkinter
cv = Canvas(root, width=w, height=h, bg='black')
cv.bind('<1>', activate_paint)
cv.pack(expand=YES, fill=BOTH)


var = StringVar()
label = Label( root, textvariable=var, relief=RAISED ,font = ("Helvetica", 50))
label.pack()

btn_save = Button(text="Reset", command=reset)
btn_save.pack()

#PIL
image1 = PIL.Image.new('L', (w, h), 'black')
draw = ImageDraw.Draw(image1)



root.mainloop()